# **Interactive Network Analysis and Visualization**  
  
This Jupyter Notebook provides a step-by-step approach to analyzing and visualizing network topology data of a power grid. Here's what you'll achieve using this notebook:  
  
1. **Load and Process Data**: Import network topology data from a CSV file for analysis.  
2. **Interactive Station Selection**: Use a dropdown menu to select a station for detailed exploration.  
3. **Network Graph Simplification**: Automatically construct and simplify the network graph for the selected station, providing the foundation for your own challenge.
4. **Visualize the Network**: Leverage visualization tools to explore the network's structure and layout.  
5. **Key Metrics and Insights**: Dynamically display important metrics, such as the number of nodes, edges, root nodes, and consumer properties.  
  
**How to Use**: Simply select a station from the dropdown menu to analyze its network structure and visualize its topology interactively.  

In [5]:
import pandas as pd
import numpy as np
import networkx as nx
import re
import pickle


from src.visualization import (
    visualize_network_topology,
    visualize_network_with_failures,
    visualize_reinforced_network # Add this import for future use
)

from src.functions import (
    build_and_simplify_network,
    find_failures_with_yearly_profile,
    suggest_grid_reinforcement,
)


# --- Start: Loading and Preparing the Data ---
# (This part is the same as before)
file_path = "../blob-account-edh/challenge-data/250903_all_stations_anon.csv"
df_full = pd.read_csv(file_path, sep=";")
print("Successfully loaded network data.")


Successfully loaded network data.


The following code provides an interactive tool for exploring network data by station. Use the dropdown menu to select a station, and the analysis results, including network properties and visualizations, will update automatically. Simply choose a station to see its network details.

In [6]:


# ==============================================================================
# --- JUPYTER-FRIENDLY INTERACTIVE WIDGETS ---
# ==============================================================================
import ipywidgets as widgets
from IPython.display import display, clear_output

# Get a list of all unique stations
all_stations = sorted(df_full['station'].unique())

# 1. Create the dropdown widget
station_dropdown = widgets.Dropdown(
    options=all_stations,
    description='Select Station:',
    value=all_stations[0] if all_stations else None,
    disabled=False,
)

# 2. Create an output widget to hold the prints and plots
output_area = widgets.Output()

# 3. Define the function that runs when the dropdown value changes
def on_station_change(change):
    selected_station = change['new']
    
    with output_area:
        clear_output(wait=True)
        print(f"--- Running analysis for station: '{selected_station}' ---")
        df_one_station = df_full[df_full['station'] == selected_station].copy()
        
        # Build the graph using the MODIFIED function
        G, consumer_props, roots = build_and_simplify_network(df_one_station)
        
        print("\n--- Results ---")
        print(f"Simplified graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
        print(f"Identified {len(roots)} root node(s): {roots}")
        print(f"Found properties for {len(consumer_props)} unique consumers.")
        
        print("\n--- Network Topology ---")
        visualize_network_topology(graph=G, root_node_ids=roots, optimize_space=True, station_name=selected_station)

# 4. Link the function to the dropdown widget
station_dropdown.observe(on_station_change, names='value')

# 5. Display the UI and run the analysis for the initial default value
print("Use the dropdown below to select a station.")
display(station_dropdown, output_area)

if station_dropdown.value:
    on_station_change({'new': station_dropdown.value})

Use the dropdown below to select a station.


Dropdown(description='Select Station:', options=('station_1', 'station_2', 'station_3', 'station_4', 'station_…

Output()